# Lab 01: Vanilla RNN with Pytorch - exercise

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/CS5242_2025_codes/labs_lecture06/lab01_vrnn_pytorch'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 153 sec ( 2.55 min)<br> 
* Time for 1 epoch on GPU : 8.4 sec w/ GeForce GTX 1080 Ti <br>

In [2]:
device= torch.device("cuda")
device= torch.device("cpu")
print(device)

if torch.cuda.is_available():
    print('cuda available with GPU:',torch.cuda.get_device_name(0))

cpu


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [3]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [4]:
bs = 20

vocab_size = 10000

### Make a recurrent net class

Step 1: Embed the discrete words into a continuous space using PyTorch’s nn.Embedding().

Step 2: Process the embedded words with a vanilla RNN layer using PyTorch’s nn.RNN().

Step 3: Project the processed tokens back into the vocabulary space by outputting a score vector over the vocabulary set with PyTorch’s nn.Linear().


In [7]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding(vocab_size, hidden_size) # COMPLETE HERE 
        self.layer2 = nn.RNN(hidden_size, hidden_size) # COMPLETE HERE 
        self.layer3 = nn.Linear(hidden_size, vocab_size)# COMPLETE HERE 

        
    def forward(self, word_seq, h_init ):
        
        g_seq               =  self.layer1(word_seq) # COMPLETE HERE 
        h_seq , h_final     =  self.layer2(g_seq, h_init) # COMPLETE HERE 
        score_seq           =  self.layer3(h_seq) # COMPLETE HERE 
        
        return score_seq,  h_final 


### Build the net. Choose the hidden size to be 150. How many parameters in total?

In [8]:
hidden_size=150

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 150)
  (layer2): RNN(150, 150)
  (layer3): Linear(in_features=150, out_features=10000, bias=True)
)
There are 3055300 (3.06 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [9]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [10]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 1
* sequence length = 35

In [11]:
criterion = nn.CrossEntropyLoss()

my_lr = 1

seq_length = 35

### Function to evaluate the network on the test set

In [12]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    
    h=h.to(device)

       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h  = net( minibatch_data, h )
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 10 passes through the training set (100 passes would reach 135 on test set)

In [13]:
start=time.time()

for epoch in range(10):
    
    # keep the learning rate to 1 during the first 4 epochs, then divide by 1.1 at every epoch
    if epoch >= 4:
        my_lr = my_lr / 1.1# COMPLETE HERE 
    
    # create a new optimizer and give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quantities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h to be the zero vector
    h = torch.zeros(1, bs, hidden_size)

    # send it to the gpu    
    h=h.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
             
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[count + 1: count + seq_length + 1]# COMPLETE HERE 
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h= h.detach()# COMPLETE HERE 
        h= h.requires_grad_()# COMPLETE HERE 
                       
        # forward the minibatch through the net        
        scores, h  = net( minibatch_data, h )
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =   scores.view(bs*seq_length, vocab_size)# COMPLETE HERE 
        minibatch_label =   minibatch_label.view(bs*seq_length)# COMPLETE HERE 
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 114.61569499969482 	 lr= 1 	 exp(loss)= 522.7883195913106
test: exp(loss) =  316.13046156294484

epoch= 1 	 time= 222.83927130699158 	 lr= 1 	 exp(loss)= 268.7440831395544
test: exp(loss) =  245.32965531932808

epoch= 2 	 time= 342.7027397155762 	 lr= 1 	 exp(loss)= 209.14582619690862
test: exp(loss) =  213.3293337275753

epoch= 3 	 time= 458.06639432907104 	 lr= 1 	 exp(loss)= 179.14004499548165
test: exp(loss) =  198.2381228300931

epoch= 4 	 time= 578.9727776050568 	 lr= 0.9090909090909091 	 exp(loss)= 158.57453035238424
test: exp(loss) =  187.14306346508658

epoch= 5 	 time= 797.573572397232 	 lr= 0.8264462809917354 	 exp(loss)= 144.00606064579398
test: exp(loss) =  173.76553036737226

epoch= 6 	 time= 1026.795194387436 	 lr= 0.7513148009015777 	 exp(loss)= 133.3734235471894
test: exp(loss) =  166.7902691376345

epoch= 7 	 time= 1245.5737018585205 	 lr= 0.6830134553650705 	 exp(loss)= 125.04229662983238
test: exp(loss) =  162.15256251122236

epoch= 8 	 time= 1468.

### Choose one sentence (taken from the test set)

In [14]:
sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# or make your own sentence.  No capital letter or punctuation allowed. Each word must be in the allowed vocabulary.
sentence6= "he was very"

# SELECT THE SENTENCE HERE
mysentence = sentence3

### Convert the sentence into a vector, then send to GPU

In [15]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[1786],
        [8705],
        [3246],
        [ 416],
        [  27],
        [  35],
        [2664],
        [ 209],
        [ 108],
        [  32],
        [3017]])


### Set the initial hidden state to zero, then run the RNN.

In [16]:
h = torch.zeros(1, 1, hidden_size)
h=h.to(device)

scores , h = net( minibatch_data , h )

### Display the network prediction for the next word

In [17]:
print(mysentence, '... \n')

utils.show_next_word(scores)

prices averaging roughly $ N a barrel higher in the third ... 

80.1%	 quarter
2.0%	 period
1.3%	 market
1.2%	 month
1.1%	 year
1.0%	 half
0.9%	 consecutive
0.5%	 week
0.4%	 fiscal
0.4%	 world
0.3%	 area
0.3%	 <eos>
0.3%	 $
0.2%	 nine
0.2%	 N
0.2%	 session
0.2%	 and
0.2%	 day
0.2%	 to
0.2%	 months
0.2%	 of
0.2%	 issue
0.2%	 game
0.1%	 <unk>
0.1%	 industry
0.1%	 time
0.1%	 third
0.1%	 wage
0.1%	 level
0.1%	 contract
